---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    G=P1_Graphs[1]
    degrees=G.degree()
    degree_values=sorted(set(degrees.values()))
    histogram=[list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
    import matplotlib.pyplot as plt
    plt.bar(degree_values,histogram)
    plt.xlabel('Degree')
    plt.ylabel('Fraction of Nodes')
    plt.show()
    print("The average shortest path is {} and the Average Clustering is {}".format(nx.average_shortest_path_length(G),
                                                                                   nx.average_clustering(G)))
    result=["PA","SW_H","SW_L","PA",'SW_H']
    return result


---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    import numpy as np
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import train_test_split
    
    nodes_data=G.nodes(data=True)
    df = pd.DataFrame(index=G.nodes())
    df['management_salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df.fillna(-10,inplace=True)
    X_test=np.array(df[(df["management_salary"]==-10)].drop("management_salary",1))
    index_test=df[(df["management_salary"]==-10)].index
    X=np.array(df[(df["management_salary"]!=-10)].drop("management_salary",1))
    y=np.array(df[(df["management_salary"]!=-10)]["management_salary"])
    LR=LogisticRegression()
    LR.fit(X,y)
    y_pred=LR.predict_proba(X_test)[:,1]
    series=pd.Series(y_pred,index=index_test)
    return series


### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [ ]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    from sklearn.linear_model import LogisticRegression
    future_connections['preferential attachment'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
    future_connections['Common Neighbors'] = future_connections.index.map(lambda city: len(list(nx.common_neighbors(G, city[0], city[1]))))
    future_connections['Resource Allocation'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
    future_connections['Adamic Adar'] = [i[2] for i in nx.adamic_adar_index(G, future_connections.index)]
    future_connections.fillna(-10,inplace=True)
    
    X_test=np.array(future_connections[(future_connections["Future Connection"]==-10)].drop("Future Connection",1))
    index_test=future_connections[(future_connections["Future Connection"]==-10)].index
    X=np.array(future_connections[(future_connections["Future Connection"]!=-10)].drop("Future Connection",1))
    y=np.array(future_connections[(future_connections["Future Connection"]!=-10)]["Future Connection"])
    LR=LogisticRegression()
    LR.fit(X,y)
    y_pred=LR.predict_proba(X_test)[:,1]
    series=pd.Series(y_pred,index=index_test)
    return series


(107, 348)    0.037351
(542, 751)    0.013262
(20, 426)     0.582790
(50, 989)     0.013516
(942, 986)    0.013793
(324, 857)    0.013481
(13, 710)     0.211960
(19, 271)     0.095213
(319, 878)    0.013605
(659, 707)    0.013288
(49, 843)     0.013685
(208, 893)    0.013349
(377, 469)    0.005796
(405, 999)    0.025495
(129, 740)    0.019631
(292, 618)    0.026670
(239, 689)    0.013623
(359, 373)    0.008268
(53, 523)     0.037401
(276, 984)    0.013672
(202, 997)    0.013726
(604, 619)    0.047506
(270, 911)    0.013618
(261, 481)    0.066876
(200, 450)    0.909954
(213, 634)    0.013184
(644, 735)    0.050729
(346, 553)    0.012568
(521, 738)    0.011442
(422, 953)    0.020755
                ...   
(672, 848)    0.013618
(28, 127)     0.967193
(202, 661)    0.012465
(54, 195)     0.999975
(295, 864)    0.013441
(814, 936)    0.013223
(839, 874)    0.013793
(139, 843)    0.013463
(461, 544)    0.010700
(68, 487)     0.010575
(622, 932)    0.013379
(504, 936)    0.016569
(479, 528) 